# 堪培拉天气预测：明天会下雨吗？

本项目旨在使用澳大利亚堪培拉的历史天气数据，通过机器学习方法预测第二天的降雨情况。我们将会经历数据清洗、预处理、模型训练、评估和优化的完整流程。

**最终模型**: XGBoost

## 1. 导入所需库
首先，我们导入项目所需的全部 Python 库。

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

## 2. 加载与探索数据
加载训练集、测试集和提交示例文件，并进行初步的数据探索。

In [ ]:
# 加载数据
train_df = pd.read_csv('./5.6_datasets/train_weather.csv', index_col='Unnamed: 0')
test_df = pd.read_csv('./5.6_datasets/test_weather.csv', index_col='Unnamed: 0')
submit_df = pd.read_csv('./5.6_datasets/submit_result.csv', index_col='Unnamed: 0')

# 快速查看数据信息，包括数据类型和非空值数量
print("--- 训练集信息 ---")
train_df.info()

print("\n--- 测试集信息 ---")
test_df.info()

## 3. 数据预处理
数据预处理是机器学习流程中的关键步骤，包括处理异常值、缺失值和对特征进行编码。

In [ ]:
# 步骤 3.1: 处理异常值
# 根据数据描述，'Cloud9am' 和 'Cloud3pm' 的取值范围应为 [0, 8]。
# 我们需要找出并删除超出此范围的异常数据行。
train_df = train_df[train_df['Cloud3pm'] != 9]
anomalous_test_indices = test_df[test_df['Cloud9am'] == 9].index
test_df = test_df.drop(anomalous_test_indices)
submit_df = submit_df.drop(anomalous_test_indices) # 提交文件也需同步删除

# 步骤 3.2: 将 'Date' 列设为索引
train_df.set_index('Date', inplace=True)
test_df.set_index('Date', inplace=True)

# 步骤 3.3: 填充缺失值 (NaN)
# 我们将特征分为数值型和分类型，并采用不同的策略进行填充。

# 识别分类型和数值型特征列
categorical_cols = train_df.select_dtypes(include='object').columns.tolist()
numerical_cols = train_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# 'RainTomorrow' 是目标变量，从特征列表中移除
categorical_cols.remove('RainTomorrow')

# 使用众数填充分类型特征的缺失值
imputer_categorical = SimpleImputer(strategy='most_frequent')
train_df[categorical_cols] = imputer_categorical.fit_transform(train_df[categorical_cols])
test_df[categorical_cols] = imputer_categorical.transform(test_df[categorical_cols])

# 使用中位数填充数值型特征的缺失值（中位数对异常值更稳健）
imputer_numerical = SimpleImputer(strategy='median')
train_df[numerical_cols] = imputer_numerical.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = imputer_numerical.transform(test_df[numerical_cols])

# 步骤 3.4: 特征编码
# 将所有分类型特征（包括目标变量）转换为数值，以便模型处理。
all_categorical_cols = categorical_cols + ['RainTomorrow']
for col in all_categorical_cols:
    le = LabelEncoder()
    # 对训练集进行拟合和转换
    train_df[col] = le.fit_transform(train_df[col])
    # 对测试集只进行转换（如果该列存在于测试集中）
    if col in test_df.columns:
        # 处理测试集中可能出现训练集未见过的新类别
        test_df[col] = test_df[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, '<unknown>')
        test_df[col] = le.transform(test_df[col])

print("数据预处理完成。检查是否还有缺失值：")
print(train_df.isnull().sum().sum())

## 4. 模型训练与评估
在这一步，我们将数据划分为训练集和验证集，然后使用 XGBoost 模型进行训练和初步评估。

In [ ]:
# 步骤 4.1: 划分特征和目标变量
X = train_df.drop('RainTomorrow', axis=1)
y = train_df['RainTomorrow']

# 步骤 4.2: 切分训练集和验证集
# 采用 80/20 的比例划分，这是常见的做法
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 步骤 4.3: 构建并训练 XGBoost 模型
model = XGBClassifier(
    n_estimators=200,          # 树的数量
    max_depth=5,               # 树的最大深度
    learning_rate=0.1,         # 学习率
    use_label_encoder=False,   # 已手动完成编码，关闭自动编码
    eval_metric='logloss',     # 评估指标
    n_jobs=-1,                 # 使用所有CPU核心
    random_state=42            # 随机种子，保证结果可复现
)

model.fit(X_train, y_train)

# 步骤 4.4: 在验证集上进行预测和评估
y_pred_val = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_val)

print(f"在验证集上的初步准确率: {accuracy:.4f}")

## 5. 模型优化 (可选)
使用 `GridSearchCV` 自动搜索最佳的超参数组合，以进一步提升模型性能。**注意：此过程在heywhale的2核8Gcpu上跑大概耗时6min。**

In [ ]:
# 定义要搜索的参数网格
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1]
}

# 创建一个新的、未训练的模型实例用于网格搜索
xgb_for_grid = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1,
    random_state=42
)

# 配置并运行 GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_for_grid, 
    param_grid=param_grid, 
    cv=3,                      # 3折交叉验证
    scoring='accuracy', 
    verbose=2
)

# 在训练数据上执行搜索
#grid_search.fit(X_train, y_train)

# 打印最佳参数和对应的分数
print(f"最佳参数: {grid_search.best_params_}")
print(f"交叉验证最高准确率: {grid_search.best_score_:.4f}")

print("网格搜索部分已注释，以节省运行时间。可取消注释以运行。")

## 6. 生成最终提交文件
使用训练好的模型对官方测试集进行预测，并生成 `submit.csv` 文件。

In [ ]:
# 使用我们训练好的初始模型 'model' 对测试集进行预测
# 如果运行了网格搜索，也可以使用 grid_search.best_estimator_ 进行预测，效果可能更好
final_predictions = model.predict(test_df)

# 将预测结果填充到提交数据框中
submit_df['RainTomorrow'] = final_predictions.astype(int)

# 保存为 CSV 文件
submit_df.to_csv('submission.csv')

print("提交文件 'submission.csv' 已成功生成。")
print(submit_df.head())